In [1]:
import pandas as pd
import requests
import json
from config import api_id_ru, api_key_ru
import time
from tqdm import tqdm

from splinter import Browser
from bs4 import BeautifulSoup
from webdriver_manager.chrome import ChromeDriverManager
# import re

# First webscrape the top 20 foods that are highest in protein from healthline.com

In [2]:
url = 'https://www.healthline.com/nutrition/20-delicious-high-protein-foods#TOC_TITLE_HDR_2'

response = requests.get(url)

# browser.visit(url)
soup = BeautifulSoup(response.text, 'html.parser')

In [3]:
#for loop tags list 1
tags1 = []

for i in range (1,10):
    want = soup.find_all('div', class_ = "css-0")[i]
    new=want.find('h2')
    tags1.append(new.text)
    
#for loop tags list 2
tags2 = []

for i in range (10,21):
    want = soup.find_all('div', class_ = "css-0")[i]
    new=want.find('h2')
    tags2.append(new.text)

In [4]:
newlst=[]
for i in tags1:
    newlst.append(i[4:])

newlst2=[]
for i in tags2:
    newlst2.append(i[5:])

In [5]:
# Joining lists
ingredients = newlst+newlst2

In [6]:
#Remove (all types) from fish
ingredients=[y.replace(' (all types)', '') for y in ingredients]

In [7]:
# Final list
ingredients

['Eggs',
 'Almonds',
 'Chicken breast',
 'Oats',
 'Cottage cheese',
 'Greek yogurt',
 'Milk',
 'Broccoli',
 'Lean beef',
 'Tuna',
 'Quinoa',
 'Whey protein supplements',
 'Lentils',
 'Ezekiel bread',
 'Pumpkin seeds',
 'Turkey breast',
 'Fish',
 'Shrimp',
 'Brussels sprouts',
 'Peanuts']

# Use this list to get 50 recipes per ingredient from Edamam API. Get recipe name, cook time, yield, calories and url where the actual recipe can be found. Store all this information in a dataframe called "recipe_df". Then save recipe_df as a csv file, called "data.csv".

In [8]:
# List of ingredients scraped from healthline, top protein sources 
ingredients
# Only interested in recipes where cook time is over 5 minutes
cook_time = '5%2B'
# Total recipes to be queried is 50
from_number = "0"
to_number = "50"

# recipe info lists
recipe_time = list()
recipe_names = list()
recipe_yields = list()
recipe_urls = list()
recipe_main = list()
recipe_cal = list()

# This loop pulls 50 recipes per ingredient
for food in tqdm(ingredients):
    url = "https://api.edamam.com/search" + "?q=" + food + "&app_id=" + api_id_ru + "&app_key=" + api_key_ru + "&time=" + cook_time + "&from=" + from_number + "&to=" + to_number
    food_request = requests.get(url)
    food_json = food_request.json()
    
    try:
        # This loop will save recipe information for each ingredient
        for x in range(0, 50):
            # Extract all recipe names for ingredient, make it the key of the dictionary
            name = food_json["hits"][x]["recipe"]["label"]
            recipe_names.append(name)
            # Extract yields for each for ingredient
            yield_amt = food_json["hits"][x]["recipe"]["yield"]
            recipe_yields.append(yield_amt)
            # Extract the url for the recipe
            recipe_url = food_json["hits"][x]["recipe"]["url"]
            recipe_urls.append(recipe_url)
            # Extract the total cook time for the recipe
            recipe_cook_time = food_json["hits"][x]["recipe"]["totalTime"]
            recipe_time.append(recipe_cook_time)
            # Extract the total calories for the recipe
            cal = food_json["hits"][x]["recipe"]["calories"]
            recipe_cal.append(cal)
            # Adding Main Ingredient
            recipe_main.append(food)
    
    except IndexError:
        recipe_names.append('N/A')
        recipe_yields.append('N/A')
        recipe_urls.append('N/A')
        recipe_time.append('N/A')
        recipe_cal.append('N/A')
        recipe_main.append('N/A')

    # Sleep in between the queries to avoid getting flagged 5 calls/min
    time.sleep(11)

100%|██████████████████████████████████████████████████████████████████████████████████| 20/20 [03:52<00:00, 11.61s/it]


In [9]:
# Check that we have the same number of items in each of the recipe information lists
print(len(recipe_names))
print(len(recipe_time))
print(len(recipe_yields))
print(len(recipe_urls))
print(len(recipe_main))
print(len(recipe_cal))

945
945
945
945
945
945


In [10]:
# Creating the dataframe for the files.
# List of recipe column names
recipe_columns = ["Recipe Name","Cook Time","Yield","Calories","Main Ingredient","URL"]
recipe_df = pd.DataFrame({"Recipe Name" : recipe_names,
                        "Cook Time" :recipe_time,
                        "Yield": recipe_yields,
                        "Calories": recipe_cal,
                        "Main Ingredient": recipe_main,
                        "URL": recipe_urls})

# Creating a dataframe for the coupons
# List of coupon column names
#coupon_columns = ["Coupon", "Exp-Date","Amount","URL","Main Ingredient"]
#coupon_df = pd.DataFrame(columns = coupon_columns)


In [11]:
recipe_df

,Recipe Name,Cook Time,Yield,Calories,Main Ingredient,URL
0,Scotch Eggs Recipe,25,4,3172.92,Eggs,http://www.seriouseats.com/recipes/2012/03/sco...
1,Moonstruck eggs,24,2,786.91,Eggs,https://food52.com/recipes/518-moonstruck-eggs
2,Eggs Benedict,60,4,2863.09,Eggs,http://www.bbc.co.uk/food/recipes/eggsbenedict...
3,Deviled eggs,17,4,302.27,Eggs,http://honestcooking.com/deviled-eggs/
4,Scrambled Eggs recipes,15,2,675.764,Eggs,http://www.tastingtable.com/entry_detail/chefs...
...,...,...,...,...,...,...
940,Thai Cucumber Salad with Peanuts,20,4,935.83,Peanuts,http://www.delish.com/cooking/recipe-ideas/rec...
941,Asian Chicken with Peanuts recipes,45,4,1825.65,Peanuts,http://www.myrecipes.com/recipe/asian-chicken-...
942,"Grilled Shrimp with Cilantro, Lime, and Peanut...",20,4,903.322,Peanuts,http://www.marthastewart.com/339717/grilled-sh...
943,Diane's Sugared Peanuts,50,4,4859.28,Peanuts,https://www.foodnetwork.com/recipes/paula-deen...


# Transform the recipe_df by renaming and correcting misspellings for recipe name

In [12]:
# Remove N/A from dataframe
food_df = recipe_df.loc[(recipe_df["Recipe Name"] != "N/A")]

In [13]:
# Remove extra words at the end of the recipe name
food_df["Recipe Name"]=food_df["Recipe Name"].replace({' recipes':''},regex=True)
food_df["Recipe Name"]=food_df["Recipe Name"].replace({' recipe':''},regex=True)
food_df["Recipe Name"]=food_df["Recipe Name"].replace({' Recipes':''},regex=True)
food_df["Recipe Name"]=food_df["Recipe Name"].replace({' Recipe':''},regex=True)

C:\Users\Kevin\anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\Kevin\anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\Kevin\anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFr

In [14]:
# Narrow down each dataframe to a single ingredient and inspect it
# Here is an example of how we did this for Quinoa

quinoa_df=food_df.loc[food_df["Main Ingredient"]=="Quinoa"]

#View full recipe names
pd.set_option('display.max_colwidth', None)
print(quinoa_df["Recipe Name"])

500                          Chicken Quinoa
501                       Plum Quinoa Cakes
502                            Quinoa cakes
503                            Basic Quinoa
504        Quinoa with Walnuts and Shallots
505                                  Quinoa
506                           Snazzy Quinoa
507                Quinoa-and-Spinach Pilaf
508              Cucumber Mint Quinoa Salad
509                        Quinoa tabbouleh
510                        Quinoa Tabbouleh
511               Quinoa Pudding with Mango
512                   Quinoa & kale muffins
513                    Quinoa Pecan Muffins
514                 Perfectly Cooked Quinoa
515                   Quinoa Breakfast Bowl
516                          Quinoa Pudding
517                        Quinoa Flatbread
518           Fried Egg Over Spinach Quinoa
519                            Sweet Quinoa
520                     Basic Boiled Quinoa
521                        Quinoa Cornbread
522                        Quino

In [15]:
# Spelling correction and/or removing unnecessary words in food_df
food_df["Recipe Name"].replace("Quinoa Tabouleh", "Quinoa Tabbouleh", inplace=True)

C:\Users\Kevin\anaconda3\envs\PythonData\lib\site-packages\pandas\core\generic.py:6746: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._update_inplace(new_data)


In [16]:
# Repeat above process for all the other main ingredients

In [17]:
#Spelling correction and/or removing unnecessary words
food_df["Recipe Name"].replace("Naomi Duguid's Fish Cakes and Fish Balls", "Fish Cakes and Fish Balls", inplace=True)
food_df["Recipe Name"].replace("MAPLE SHEER FISH", "Maple Sheer Fish", inplace=True)
food_df["Recipe Name"].replace("Escovitched Fish", "Escovitch Fish", inplace=True)
food_df["Recipe Name"].replace("Boiled Peanuts (五香花生) recipes", "Boiled Peanuts", inplace=True)
food_df["Recipe Name"].replace("Diane's Sugared Peanuts", "Sugared Peanuts", inplace=True)
food_df["Recipe Name"].replace("Honey Greek Yogurt Fillo Cups", "Honey Greek Yogurt Phyllo Cups", inplace=True)
food_df["Recipe Name"].replace("Cook the Book: Chickpea Sauté with Greek Yogurt", "Chickpea Sauté with Greek Yogurt", inplace=True)
food_df["Recipe Name"].replace("Banana Split No-Bake Greek Yogurt Cheesecake From 'Seriously Delish'", "Banana Split No-Bake Greek Yogurt Cheesecake", inplace=True)
food_df["Recipe Name"].replace("Fruit and Milk Smoothy", "Fruit and Milk Smoothie", inplace=True)
food_df["Recipe Name"]=food_df["Recipe Name"].replace({'Chilli':'chili'},regex=True)
food_df["Recipe Name"]=food_df["Recipe Name"].replace({'chilli':'chili'},regex=True)
food_df["Recipe Name"].replace('Devilled Eggs', 'Deviled Eggs', inplace=True)
food_df["Recipe Name"].replace('Eggs Florentine I', 'Eggs Florentine', inplace=True)

C:\Users\Kevin\anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()
C:\Users\Kevin\anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':


In [18]:
food_df.to_csv('../../datasets/data.csv', index = False)

In [19]:
food_df

,Recipe Name,Cook Time,Yield,Calories,Main Ingredient,URL
0,Scotch Eggs,25,4,3172.92,Eggs,http://www.seriouseats.com/recipes/2012/03/scotch-eggs-recipe.html
1,Moonstruck eggs,24,2,786.91,Eggs,https://food52.com/recipes/518-moonstruck-eggs
2,Eggs Benedict,60,4,2863.09,Eggs,http://www.bbc.co.uk/food/recipes/eggsbenedict_9435
3,Deviled eggs,17,4,302.27,Eggs,http://honestcooking.com/deviled-eggs/
4,Scrambled Eggs,15,2,675.764,Eggs,http://www.tastingtable.com/entry_detail/chefs_recipes/16394/Scrambled_Eggs_Recipe.htm
...,...,...,...,...,...,...
940,Thai Cucumber Salad with Peanuts,20,4,935.83,Peanuts,http://www.delish.com/cooking/recipe-ideas/recipes/a20711/thai-cucumber-salad-peanuts-recipe-fw0313/
941,Asian Chicken with Peanuts,45,4,1825.65,Peanuts,http://www.myrecipes.com/recipe/asian-chicken-with-peanuts
942,"Grilled Shrimp with Cilantro, Lime, and Peanuts",20,4,903.322,Peanuts,http://www.marthastewart.com/339717/grilled-shrimp-with-cilantro-lime-and-pe
943,Sugared Peanuts,50,4,4859.28,Peanuts,https://www.foodnetwork.com/recipes/paula-deen/dianes-sugared-peanuts-recipe-1940230
